In [1]:

import pickle, string, numpy, getopt, sys, random, time, re, pprint, gc, resource
import pandas as pd
import onlineldavb
import scrapeWoS
import gensim
import nltk
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import sys
import time
from multiprocess import Pool
import django
from functools import partial
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from time import time
from django.utils import timezone
from scipy.sparse import csr_matrix, find
import numpy as np
import psycopg2

#conn = psycopg2.connect("dbname=tmv_app user=tmv password=topicmodels")

sys.stdout.flush()

# import file for easy access to browser database
sys.path.append('/home/galm/software/tmv/BasicBrowser/')

# sys.path.append('/home/max/Desktop/django/BasicBrowser/')
import db as db
from tmv_app.models import *
from scoping.models import Doc
from django.db import connection, transaction
cursor = connection.cursor()

def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i

def f_gamma(docs,gamma,docsizes,docUTset,topic_ids):
    dts = []
    for d in docs:
        dt = DocTopic(
            doc_id = docUTset[gamma[0][d]],
            topic_id = topic_ids[gamma[1][d]],
            score = gamma[2][d],
            scaled_score = gamma[2][d] / docsizes[gamma[0][d]],
            run_id=run_id
        )
        dts.append(dt)
    return dts

def f_gamma2(docs,gamma,docsizes,docUTset,topic_ids):
    vl = []
    for d in docs:
        dt = (
            docUTset[gamma[0][d]],
            topic_ids[gamma[1][d]],
            gamma[2][d],
            gamma[2][d] / docsizes[gamma[0][d]],
            run_id
        )
        vl.append(dt)
    return vl

def f_lambda(t,m,v_ids,t_ids):
    tt = TopicTerm(
        term_id = v_ids[m[1][t]],
        topic_id = t_ids[m[0][t]],
        score = m[2][t],
        run_id = run_id
    )
    return tt

def tokenize(text):
    transtable = {ord(c): None for c in string.punctuation + string.digits}
    tokens = nltk.word_tokenize(text.translate(transtable))
    tokens = [i for i in tokens if len(i) > 2]
    return tokens


def add_features(title):
    django.db.connections.close_all()
    term, created = Term.objects.get_or_create(title=title)
    term.run_id.add(run_id)
    django.db.connections.close_all()
    return term.pk

def update_features(id):
    django.db.connections.close_all()
    term = Term.objects.get(pk=id)
    term.run_id.add(run_id)
    django.db.connections.close_all()
    return term.pk

def bulk_create_par(dts):
    DocTopic.objects.bulk_create(dts)

In [3]:
K = 20 
ng = 1
n_features = 10000
limit = 1000

n_samples = 1000

#############################################
## STOPWORDS
stoplist = set(nltk.corpus.stopwords.words("english"))
stoplist.add('elsevier')
stoplist.add('rights')
stoplist.add('reserved')
stoplist.add('john')
stoplist.add('wiley')
stoplist.add('sons')
stoplist.add('copyright')

#docs = Doc.objects.filter(query=893,content__iregex='\w').values('UT','title','content')
docs = Doc.objects.filter(query=365,content__iregex='\w')
docs = docs#.values('UT','content')
if limit is not False:
    docs = docs[:limit]

print("{} documents found".format(len(docs)))


abstracts = [re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",x.content)[0] for x in docs.iterator()]
abstracts = [x.split("Published by Elsevier")[0] for x in abstracts]
abstracts = [x.split("Copyright (C)")[0] for x in abstracts]
abstracts = [re.split("\. \(C\) [1-2][0-9]{3} ",x)[0] for x in abstracts]
docsizes = [len(x) for x in abstracts]
ids = [x.UT for x in docs.iterator()]

del docs
class snowball_stemmer(object):
    def __init__(self):
        self.stemmer = SnowballStemmer("english")
    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in tokenize(doc)]


#############################################
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=10,
                                   max_features=n_features,
                                   ngram_range=(ng,ng),
                                   tokenizer=snowball_stemmer(),
                                   stop_words=stoplist)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(abstracts)
print("done in %0.3fs." % (time() - t0))

del abstracts

gc.collect()

vocab = tfidf_vectorizer.get_feature_names()

print(len(vocab))
nmf = NMF(n_components=K, random_state=1,
                  alpha=.1, l1_ratio=.5).fit(tfidf)


1000 documents found
Extracting tf-idf features for NMF...
done in 5.212s.
1524


In [50]:
gamma =  nmf.transform(tfidf)
sgamma = csr_matrix(nmf.transform(tfidf))
print(sgamma.size)
print(sgamma)

keep = sgamma[sgamma >= 0.1]
print(keep.shape)

#keep = np.ravel(keep)

print(keep.size)
print(keep)

print(keep.shape)



sgammaf = sgamma[keep,:]
print(sgammaf.size)
print(sgammaf)

4881
  (0, 0)	0.0435509396758
  (0, 8)	0.15371058451
  (0, 12)	0.0373727662533
  (0, 18)	0.018054114236
  (1, 0)	0.0177038563869
  (1, 1)	0.0249310170922
  (1, 3)	0.0520638073836
  (1, 5)	0.276867245521
  (2, 0)	0.0652269832473
  (2, 2)	0.00117803665191
  (2, 3)	0.076796318373
  (2, 4)	0.0269326241916
  (2, 11)	0.0635907859801
  (3, 0)	0.058040644977
  (3, 1)	0.022098140311
  (3, 2)	0.0827347406459
  (3, 3)	0.0261200984619
  (3, 11)	0.0309097741662
  (4, 0)	0.0182891462227
  (4, 1)	0.0666940164489
  (4, 8)	0.031423320385
  (4, 10)	0.0944054889213
  (5, 0)	0.0344191866075
  (5, 1)	0.00581012174244
  (5, 6)	0.180190629248
  :	:
  (994, 4)	0.0402726606849
  (994, 9)	0.0568203576577
  (995, 0)	0.00739357468587
  (995, 1)	0.173558314388
  (995, 19)	0.044328822373
  (996, 0)	0.0503265418069
  (996, 5)	0.0453984777261
  (996, 9)	0.0737961606852
  (996, 10)	0.131096187113
  (997, 0)	0.00353038829558
  (997, 1)	0.0130742281523
  (997, 6)	0.00225661422112
  (997, 8)	0.0453740063097
  (997, 10)	0

IndexError: index returns 3-dim structure

In [34]:
sgamma

<1000x20 sparse matrix of type '<class 'numpy.float64'>'
	with 4881 stored elements in Compressed Sparse Row format>

In [40]:
keep

array([ <1000x20 sparse matrix of type '<class 'numpy.bool_'>'
	with 827 stored elements in Compressed Sparse Row format>], dtype=object)

In [36]:
keep[0]

<1000x20 sparse matrix of type '<class 'numpy.bool_'>'
	with 827 stored elements in Compressed Sparse Row format>

In [37]:
keep[0][1]

<1x20 sparse matrix of type '<class 'numpy.bool_'>'
	with 1 stored elements in Compressed Sparse Row format>

In [41]:
keep[0].nonzero()

(array([  0,   1,   5,   6,   7,   8,   8,  10,  12,  13,  15,  15,  17,
         17,  18,  19,  21,  23,  24,  24,  26,  26,  26,  27,  29,  30,
         31,  33,  35,  36,  37,  37,  38,  39,  41,  44,  45,  47,  48,
         49,  50,  51,  53,  55,  55,  56,  58,  58,  59,  61,  63,  64,
         64,  65,  66,  68,  74,  74,  74,  75,  76,  76,  78,  79,  80,
         81,  82,  87,  89,  89,  90,  91,  93,  93,  95,  98,  99, 100,
        102, 102, 103, 104, 105, 105, 108, 109, 110, 111, 111, 113, 114,
        118, 119, 120, 121, 122, 122, 124, 125, 126, 126, 128, 129, 130,
        131, 132, 134, 135, 135, 135, 136, 137, 139, 140, 141, 143, 143,
        144, 147, 147, 148, 149, 151, 156, 157, 159, 160, 160, 161, 162,
        165, 167, 169, 171, 172, 173, 173, 173, 174, 175, 179, 180, 181,
        184, 186, 187, 188, 189, 190, 191, 192, 193, 193, 195, 196, 197,
        199, 200, 202, 204, 204, 209, 209, 211, 213, 214, 215, 215, 217,
        218, 221, 222, 224, 225, 227, 228, 228, 230